In [ ]:
# OID = 2226832103641712942
# OID = 1053090246851297631
# OID = 2226880482153333268
# OID = 2226928860664971934
# OID = 2226792521223122099
# OID = 2528742163131035981
# OID = 2226748540758019375
# OID = 2226748540758003742
# OID = 2132955800862215099
# OID = 2133004179373834781
OID = 2226792521223124930

In [ ]:
# %pip install -U lsdb
# %pip install 'matplotlib>=3.10'

In [ ]:
from pathlib import Path

release = "v29_0_0_rc5"
hats_path = Path("/sdf/data/rubin/shared/lsdb_commissioning/hats/") / release
# list dir
print(list(map(str, hats_path.iterdir())))

obj_lc_path = hats_path / "object_lc"

In [ ]:
from astropy.coordinates import SkyCoord, EarthLocation
from astropy.time import Time


lsst_location = EarthLocation.of_site("LSST")


def add_helio_mjd(df):
    coord = SkyCoord(ra=df["lc.coord_ra"], dec=df["lc.coord_dec"], unit="deg")
    time = Time(df["lc.midpointMjdTai"], format="mjd", scale="tai")
    helio_time = time + time.light_travel_time(coord, "heliocentric", location=lsst_location)
    df["lc.helioMjd"] = helio_time.mjd
    return df

In [ ]:
# Load the Forced Source + MJD Table
from lsdb import read_hats
from nested_pandas import NestedDtype

filters = [("objectId", "==", OID)]

obj_lc = read_hats(
    obj_lc_path,
    filters=filters,
).map_partitions(
    lambda df: df.assign(
        lc=df["objectForcedSource"].astype(
                NestedDtype.from_pandas_arrow_dtype(df.dtypes["objectForcedSource"])
        ),
    ).drop(columns=["objectForcedSource"]),
).map_partitions(
    add_helio_mjd,
)

In [ ]:
import pandas as pd; pd.set_option('display.max_columns', None)
ndf = obj_lc.compute()
ndf = ndf.query(
    "~lc.psfFlux_flag"
    " and ~lc.pixelFlags_suspect"
    " and ~lc.pixelFlags_saturated"
    " and ~lc.pixelFlags_cr"
    " and ~lc.pixelFlags_bad"
).sort_values(
    "lc.midpointMjdTai"
)
display(ndf)
data = ndf.iloc[0]
display(data)

In [ ]:
for band in 'ugrizy':
    flux_over_error = data[f"{band}_psfFlux"] / data[f"{band}_psfFluxErr"]
    print(f"{band} flux/error = {flux_over_error:.2f}")

# SCIENCE

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from astropy.timeseries import LombScargleMultiband

COLORS = {'u': '#0c71ff', 'g': '#49be61', 'r': '#c61c00',
          'i': '#ffc200', 'z': '#f341a2', 'y': '#5d0000'}

lc = data.lc.query('psfFlux / psfFluxErr > 3 and ~psfFlux_flag')

mag = -2.5 * np.log10(lc['psfFlux']) + 31.4
magerr = 2.5 / np.log(10) * lc['psfFluxErr'] / lc['psfFlux']

plt.figure()
for b in 'ugrizy':
    i = lc['band'] == b
    if not np.any(i):
        continue
    mean_mag = np.average(mag[i], weights=1/magerr[i]**2)
    plt.errorbar(lc['midpointMjdTai'][i], mag[i] - mean_mag, magerr[i], fmt="o", alpha=0.3, color=COLORS[b], label=f'{b} $-${mean_mag:.2f}')
plt.gca().invert_yaxis()
plt.title(f'{OID}')
# plt.xlim(60640.19, 60640.20)
plt.xlabel('MJD')
plt.ylabel('magnitude - average magnitude')
plt.legend()

In [ ]:
def extract_period_multiband_astropy(band, t, y, yerr, plot=True):
    freq, power = LombScargleMultiband(t, y, band, yerr).autopower(samples_per_peak=3000, nyquist_factor=1)
    idx_period = np.argmax(power)
    period = 1 / freq[idx_period]
    
    if plot:
        plt.plot(freq, power)
        plt.vlines(1/period, 0, power.max(), lw=2, ls='--', color='red')
    
    return period

period = extract_period_multiband_astropy(lc['band'], lc['helioMjd'], mag, magerr)
# period = extract_period_multiband_astropy(lc['band'], lc['helioMjd'], lc['psfFlux'], lc['psfFluxErr'])

phase = lc['helioMjd'] % period / period

plt.figure()
for b in 'ugrizy':
    i = lc['band'] == b
    mean_mag = np.average(mag[i], weights=1/magerr[i]**2)
    plt.errorbar(phase[i], mag[i] - mean_mag, magerr[i], fmt="o", alpha=0.3, color=COLORS[b], label=f'{b} $-${mean_mag:.2f}')
plt.gca().invert_yaxis()
plt.title(f'$P = {period:.6f}$ days')
plt.xlim(0, 1)
plt.xlabel('phase')
plt.ylabel('magnitude - average magnitude')
plt.legend()

In [ ]:
plt.figure()
for b in 'gz':
    i = lc['band'] == b
    mean_mag = np.average(mag[i], weights=1/magerr[i]**2)
    plt.errorbar(phase[i], mag[i] - mean_mag, magerr[i], fmt="o", alpha=0.1, color=COLORS[b], label=f'{b} $-${mean_mag:.2f}')
plt.gca().invert_yaxis()
plt.title(f'$P = {period:.6f}$ days')
plt.xlim(0, 1)
plt.xlabel('phase')
plt.ylabel('magnitude - average magnitude')
plt.legend()